# DQX Demo

In [0]:
%pip install databricks-labs-dqx==0.8.0

In [0]:
dbutils.library.restartPython()

In [0]:
# =====================================================================================
# CELL 1 — demo_spec.py  (Schemas + MEGASPEC + rule-class stubs + table directory)
# =====================================================================================
# Purpose: single source of truth for table names, schemas, row targets, and knobs.
# =====================================================================================

from pyspark.sql.types import (
    StructType, StructField,
    StringType, BooleanType, DateType, TimestampType, DecimalType
)

# ──────────────────────────────────────────────────────────────────────────────
# Globals / conventions
# ──────────────────────────────────────────────────────────────────────────────
DQX_CATALOG        = "dq_dev"
DQX_SCHEMA         = "dqx"                     # base schema for demo tables
QUARANTINE_TABLE   = f"{DQX_CATALOG}.{DQX_SCHEMA}.demo_quarantine"  # sink for ERROR rows
DQ_OUTPUT_COLUMNS  = ["warning", "error"]      # columns your runner appends

ROW_TARGETS = {
    f"{DQX_SCHEMA}.demo_employee":   2_000,
    f"{DQX_SCHEMA}.demo_customer":   1_000,
    f"{DQX_SCHEMA}.demo_project":      600,
    f"{DQX_SCHEMA}.demo_timesheet": 350_000,
    f"{DQX_SCHEMA}.demo_expense":   120_000,
}

# Canonical table names (full paths) + groupings you can use anywhere
TABLES = {
    "schema": DQX_SCHEMA,
    "employee":   f"{DQX_SCHEMA}.demo_employee",
    "customer":   f"{DQX_SCHEMA}.demo_customer",
    "project":    f"{DQX_SCHEMA}.demo_project",
    "timesheet":  f"{DQX_SCHEMA}.demo_timesheet",
    "expense":    f"{DQX_SCHEMA}.demo_expense",
    "quarantine": QUARANTINE_TABLE,
    # Groupings
    "sources": [
        f"{DQX_SCHEMA}.demo_employee",
        f"{DQX_SCHEMA}.demo_customer",
        f"{DQX_SCHEMA}.demo_project",
        f"{DQX_SCHEMA}.demo_timesheet",
        f"{DQX_SCHEMA}.demo_expense",
    ],
    "facts": [
        f"{DQX_SCHEMA}.demo_timesheet",
        f"{DQX_SCHEMA}.demo_expense",
    ],
    "dims": [
        f"{DQX_SCHEMA}.demo_employee",
        f"{DQX_SCHEMA}.demo_customer",
        f"{DQX_SCHEMA}.demo_project",
    ],
    "all": [
        f"{DQX_SCHEMA}.demo_employee",
        f"{DQX_SCHEMA}.demo_customer",
        f"{DQX_SCHEMA}.demo_project",
        f"{DQX_SCHEMA}.demo_timesheet",
        f"{DQX_SCHEMA}.demo_expense",
        QUARANTINE_TABLE,
    ],
}

# ──────────────────────────────────────────────────────────────────────────────
# Spark Structured Schemas
# ──────────────────────────────────────────────────────────────────────────────

demo_employee_schema = StructType([
    StructField("employee_id",        StringType(),  False),
    StructField("full_name",          StringType(),  False),
    StructField("department",         StringType(),  False),  # Consulting, Audit, Tax, IT, Ops
    StructField("role",               StringType(),  False),  # Engineer, Analyst, Consultant, Manager, Support
    StructField("cost_center",        StringType(),  True),   # CC-####
    StructField("employment_status",  StringType(),  False),  # Active, Leave, Terminated
    StructField("hire_date",          DateType(),    False),
    StructField("termination_date",   DateType(),    True),
    StructField("work_email",         StringType(),  True),
    StructField("country_code",       StringType(),  True),
    # DQ columns appended later: warning, error
])

demo_customer_schema = StructType([
    StructField("customer_id",            StringType(),  False),
    StructField("customer_name",          StringType(),  False),
    StructField("industry",               StringType(),  False),  # Technology, Healthcare, Finance, Retail, Manufacturing
    StructField("country_code",           StringType(),  True),   # ISO-3166 alpha-2
    StructField("status",                 StringType(),  False),  # Active, Prospect, Inactive
    StructField("onboarding_date",        DateType(),    False),
    StructField("primary_contact_email",  StringType(),  True),
    StructField("registration_number",    StringType(),  True),   # national reg / tax-like id
])

demo_project_schema = StructType([
    StructField("project_id",          StringType(),       False),
    StructField("customer_id",         StringType(),       False),  # FK -> dqx.demo_customer
    StructField("project_name",        StringType(),       False),
    StructField("status",              StringType(),       False),  # Planned, Active, OnHold, Closed
    StructField("start_date",          DateType(),         False),
    StructField("end_date",            DateType(),         True),
    StructField("manager_employee_id", StringType(),       True),   # FK -> dqx.demo_employee
    StructField("budget_amount",       DecimalType(18, 2), True),
    StructField("billing_model",       StringType(),       False),  # T&M, Fixed, Retainer
])

demo_timesheet_schema = StructType([
    StructField("timesheet_id",  StringType(),       False),
    StructField("employee_id",   StringType(),       False),  # FK -> dqx.demo_employee
    StructField("project_id",    StringType(),       False),  # FK -> dqx.demo_project
    StructField("work_date",     DateType(),         False),
    StructField("hours_worked",  DecimalType(5, 2),  False),  # 0–24
    StructField("work_type",     StringType(),       False),  # Billable, NonBillable, Admin
    StructField("source_system", StringType(),       False),  # Workday, Jira, CSV
    StructField("created_ts",    TimestampType(),    False),
])

demo_expense_schema = StructType([
    StructField("expense_id",       StringType(),       False),
    StructField("employee_id",      StringType(),       False),  # FK -> dqx.demo_employee
    StructField("project_id",       StringType(),       True),   # FK -> dqx.demo_project (nullable)
    StructField("expense_date",     DateType(),         False),
    StructField("category",         StringType(),       False),  # Meals, Travel, Supplies, Software, Other
    StructField("amount",           DecimalType(18, 2), False),
    StructField("currency_code",    StringType(),       False),  # ISO-4217
    StructField("merchant",         StringType(),       True),
    StructField("receipt_attached", BooleanType(),      False),
    StructField("submission_ts",    TimestampType(),    False),
])

# Handy schema map (full paths)
SCHEMAS_BY_TABLE = {
    TABLES["employee"]:  demo_employee_schema,
    TABLES["customer"]:  demo_customer_schema,
    TABLES["project"]:   demo_project_schema,
    TABLES["timesheet"]: demo_timesheet_schema,
    TABLES["expense"]:   demo_expense_schema,
}

TABLE_ORDER = [
    TABLES["employee"],
    TABLES["customer"],
    TABLES["project"],
    TABLES["timesheet"],
    TABLES["expense"],
]

# ──────────────────────────────────────────────────────────────────────────────
# MEGASPEC (nested, readable). Map this to dbldatagen/ddgen in your builder.
# Fully qualified refs use the dqx schema.
# ──────────────────────────────────────────────────────────────────────────────

MEGASPEC = {
    "schema": DQX_SCHEMA,
    "quarantine_table": QUARANTINE_TABLE,
    "dq_output_columns": DQ_OUTPUT_COLUMNS,          # appended by your rule runner
    "flags": {
        "enforce_project_window": True,              # error if outside project dates
        "allow_weekend_billable": "warn",            # 'warn' | 'error' | 'off'
    },
    "knobs": {
        "receipt_threshold": 75.00,                  # USD; >= threshold requires receipt
        "meal_limit": 150.00,                        # warn if Meals > limit
        "travel_limit": 500.00,                      # warn if Travel > limit
        "max_hours_per_day_error": 24.0,             # error if > this
        "hi_hours_warn": 12.0,                       # optional warn if > this
        "duplicate_timesheet_window_days": 0,        # same-day dupes
        "duplicate_expense_exact_match": True,       # dup key = (emp, merchant, date, amount)
        "valid_currency_codes": ["USD","CAD","MXN","GBP","INR"],
    },
    "tables": {
        f"{DQX_SCHEMA}.demo_employee": {
            "rows": ROW_TARGETS[f"{DQX_SCHEMA}.demo_employee"],
            "cols": {
                "employee_id": {"type":"sequence","prefix":"E","start":1001},
                "full_name": {"type":"faker","provider":"name"},
                "department": {"type":"choice","values":["Consulting","Audit","Tax","IT","Ops"],"weights":[0.35,0.20,0.20,0.15,0.10]},
                "role": {"type":"choice","values":["Engineer","Analyst","Consultant","Manager","Support"]},
                "cost_center": {"type":"pattern","format":"CC-{0000-9999}","null_rate":0.02},
                "employment_status": {"type":"choice","values":["Active","Leave","Terminated"],"weights":[0.90,0.03,0.07]},
                "hire_date": {"type":"date","start":"2018-01-01","end":"2025-08-01"},
                "termination_date": {"type":"conditional_date","when":"employment_status == 'Terminated'","offset_from":"hire_date","min_days":30,"max_days":2500,"null_rate":0.08},
                "work_email": {"type":"template","template":"{first}.{last}@company.com","invalid_rate":0.03},
                "country_code": {"type":"choice","values":["US","CA","MX","GB","IN"],"invalid_rate":0.005}
            },
            "inject": {"EMP_TERM_DATE_BEFORE_HIRE": 0.008}
        },

        f"{DQX_SCHEMA}.demo_customer": {
            "rows": ROW_TARGETS[f"{DQX_SCHEMA}.demo_customer"],
            "cols": {
                "customer_id": {"type":"sequence","prefix":"C","start":5001},
                "customer_name": {"type":"faker","provider":"company"},
                "industry": {"type":"choice","values":["Technology","Healthcare","Finance","Retail","Manufacturing"],"weights":[0.30,0.20,0.20,0.20,0.10]},
                "country_code": {"type":"choice","values":["US","CA","MX","GB","IN"],"invalid_rate":0.005},
                "status": {"type":"choice","values":["Active","Prospect","Inactive"],"weights":[0.75,0.15,0.10]},
                "onboarding_date": {"type":"date","start":"2019-01-01","end":"2025-08-01"},
                "primary_contact_email": {"type":"faker","provider":"email","invalid_rate":0.02},
                "registration_number": {"type":"pattern","format":"RN-{00000000-99999999}","dupe_rate_active":0.01}
            }
        },

        f"{DQX_SCHEMA}.demo_project": {
            "rows": ROW_TARGETS[f"{DQX_SCHEMA}.demo_project"],
            "cols": {
                "project_id": {"type":"sequence","prefix":"P","start":10001},
                "customer_id": {"type":"fk","ref": f"{DQX_SCHEMA}.demo_customer.customer_id","null_rate":0.01},
                "project_name": {"type":"template","template":"Project {seq}"},
                "status": {"type":"choice","values":["Planned","Active","OnHold","Closed"],"weights":[0.15,0.55,0.10,0.20]},
                "start_date": {"type":"date","start":"2020-01-01","end":"2025-03-01"},
                "end_date": {"type":"date_or_null","null_rate":0.35,"min_from":"start_date","min_days":30,"max_days":1200},
                "manager_employee_id": {"type":"fk","ref": f"{DQX_SCHEMA}.demo_employee.employee_id","null_rate":0.03},
                "budget_amount": {"type":"decimal","min":10_000.00,"max":2_000_000.00,"skew":"right"},
                "billing_model": {"type":"choice","values":["T&M","Fixed","Retainer"],"weights":[0.55,0.35,0.10]}
            },
            "inject": {"PROJ_END_BEFORE_START":0.007,"PROJ_BUDGET_NONPOSITIVE":0.004}
        },

        f"{DQX_SCHEMA}.demo_timesheet": {
            "rows": ROW_TARGETS[f"{DQX_SCHEMA}.demo_timesheet"],
            "cols": {
                "timesheet_id": {"type":"uuid"},
                "employee_id": {"type":"fk","ref": f"{DQX_SCHEMA}.demo_employee.employee_id"},
                "project_id": {"type":"fk","ref": f"{DQX_SCHEMA}.demo_project.project_id"},
                "work_date": {"type":"date","start":"2024-01-01","end":"2025-08-10","weekday_bias":0.85},
                "hours_worked": {"type":"decimal","min":0.00,"max":12.00,"step":0.25,"tail_spikes":[20.00,24.00],"tail_rate":0.004},
                "work_type": {"type":"choice","values":["Billable","NonBillable","Admin"],"weights":[0.70,0.25,0.05]},
                "source_system": {"type":"choice","values":["Workday","Jira","CSV"],"weights":[0.70,0.20,0.10]},
                "created_ts": {"type":"timestamp_near","base":"work_date","min_offset_hours":0,"max_offset_hours":72}
            },
            "inject": {"TS_FUTURE_DATE":0.003,"TS_DUP_EMP_PROJ_DAY":0.006,"TS_NEG_OR_GT24":0.003}
        },

        f"{DQX_SCHEMA}.demo_expense": {
            "rows": ROW_TARGETS[f"{DQX_SCHEMA}.demo_expense"],
            "cols": {
                "expense_id": {"type":"uuid"},
                "employee_id": {"type":"fk","ref": f"{DQX_SCHEMA}.demo_employee.employee_id"},
                "project_id": {"type":"fk","ref": f"{DQX_SCHEMA}.demo_project.project_id","null_rate":0.25},
                "expense_date": {"type":"date","start":"2024-01-01","end":"2025-08-10","weekend_bias":0.20},
                "category": {"type":"choice","values":["Meals","Travel","Supplies","Software","Other"],"weights":[0.35,0.25,0.20,0.10,0.10]},
                "amount": {"type":"decimal","min":5.00,"max":5_000.00,"skew":"right","round_dollar_rate":0.10},
                "currency_code": {"type":"choice","values":["USD","CAD","MXN","GBP","INR"],"invalid_rate":0.002},
                "merchant": {"type":"choice","values":["Uber","Lyft","Delta","AA","Staples","BestBuy","Amazon","LocalCafe","HotelCo","SoftwareCo"]},
                "receipt_attached": {"type":"boolean","true_rate":0.92},
                "submission_ts": {"type":"timestamp_near","base":"expense_date","min_offset_hours":0,"max_offset_hours":240}
            },
            "inject": {"EXP_DUP":0.008,"EXP_NO_RECEIPT_OVER_75":0.010,"EXP_OUT_OF_POLICY":0.020}
        }
    }
}

# ──────────────────────────────────────────────────────────────────────────────
# Rule classes (stubs). Define the .rules() bodies in a separate module or here.
# Each .rules() should return list[dict]:
#   { "id": str, "level": "error"|"warn", "expr": "<Spark SQL>", "message": str, ... }
# ──────────────────────────────────────────────────────────────────────────────

class BaseRuleSet:
    """Interface for a rule set; implement .rules() to return a list of rule dicts"""
    def rules(self):
        raise NotImplementedError("Implement .rules() to return a list of rule dicts")

class EmployeeRules(BaseRuleSet):
    """dqx.demo_employee — suggested IDs: EMP_TERM_DATE_MISSING, EMP_TERM_DATE_BEFORE_HIRE, EMP_EMAIL_DOMAIN_WARN, EMP_COUNTRY_CODE_WARN"""
    pass

class CustomerRules(BaseRuleSet):
    """dqx.demo_customer — suggested IDs: CUST_REG_DUP_ACTIVE, CUST_EMAIL_WARN, CUST_COUNTRY_CODE_WARN"""
    pass

class ProjectRules(BaseRuleSet):
    """dqx.demo_project — suggested IDs: PROJ_END_BEFORE_START, PROJ_BUDGET_NONPOSITIVE, PROJ_FK_CUSTOMER_MISSING, PROJ_MANAGER_FK_WARN, PROJ_CLOSED_END_NULL_WARN"""
    pass

class TimesheetRules(BaseRuleSet):
    """dqx.demo_timesheet — suggested IDs: TS_NEG_OR_GT24, TS_HI_HOURS_WARN, TS_FUTURE_DATE, TS_EMP_STATUS_ERROR, TS_DUP_EMP_PROJ_DAY, TS_OUTSIDE_PROJECT_WINDOW, TS_WEEKEND_BILLABLE"""
    pass

class ExpenseRules(BaseRuleSet):
    """dqx.demo_expense — suggested IDs: EXP_DUP, EXP_NO_RECEIPT_OVER_T, EXP_OOP, EXP_BAD_CCY, EXP_EMP_STATUS_ERROR, EXP_OUTSIDE_PROJECT_WINDOW"""
    pass

# Map table → rule class so a driver can instantiate dynamically
RULE_CLASSES_BY_TABLE = {
    TABLES["employee"]:  EmployeeRules,
    TABLES["customer"]:  CustomerRules,
    TABLES["project"]:   ProjectRules,
    TABLES["timesheet"]: TimesheetRules,
    TABLES["expense"]:   ExpenseRules,
}

print(f"\n[SPEC LOADED] schema={DQX_SCHEMA}  sources={len(TABLES['sources'])}  quarantine={TABLES['quarantine']}")
print(f"Row targets: " + ", ".join([f"{k.split('.')[-1]}={v:,}" for k,v in ROW_TARGETS.items()]))

# COMMAND ----------
# =====================================================================================
# CELL 2 — Driver setup (imports, DB/schema, quarantine table)
# =====================================================================================

from pyspark.sql import functions as F
from pyspark.sql import Window
from uuid import uuid4

# Pull in the spec from Cell 1 (already in-memory if same notebook)
from __main__ import (
    DQX_SCHEMA,
    TABLES,
    ROW_TARGETS,
    SCHEMAS_BY_TABLE,
    MEGASPEC,
    RULE_CLASSES_BY_TABLE,
    QUARANTINE_TABLE
)

def banner(msg):
    print("\n" + "═"*88)
    print(f" {msg}")
    print("═"*88)

banner("SETUP: Create schema and quarantine table")

spark.sql(f"CREATE DATABASE IF NOT EXISTS {DQX_SCHEMA}")
spark.sql(f"USE {DQX_SCHEMA}")

run_id = str(uuid4())
print(f"Using schema: {DQX_SCHEMA}")
print(f"Run ID: {run_id}")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {QUARANTINE_TABLE} (
  _source_table     STRING,
  _rule_id          STRING,
  _level            STRING,
  _reason           STRING,
  _run_id           STRING,
  _event_ts         TIMESTAMP,
  _row_payload_json STRING
) USING delta
""")
print(f"Quarantine table ready: {QUARANTINE_TABLE}")

# COMMAND ----------
# =====================================================================================
# CELL 3 — Helpers (small utilities for deterministic, lightweight data gen)
# =====================================================================================

banner("HELPERS: small UDF-like helpers for datagen")

def seq_id(prefix: str, start: int):
    return F.concat(F.lit(prefix), F.lpad((F.col("_rn") + F.lit(start)).cast("string"), 4, "0"))

def choice_expr(options, weights=None, seed_col="_rand"):
    if not weights:
        weights = [1.0/len(options)]*len(options)
    cum = []
    s = 0.0
    for w in weights:
        s += float(w); cum.append(s)
    x = F.col(seed_col)
    expr = None
    for i, val in enumerate(options):
        cond = (x <= cum[i])
        expr = (F.when(cond, F.lit(val)) if expr is None
                else expr.otherwise(F.when(cond, F.lit(val))))
    return expr.otherwise(F.lit(options[-1]))

def rand_date(start: str, end: str, rand_expr="rand()"):
    # random date between start and end inclusive
    days = F.datediff(F.lit(end), F.lit(start))
    return F.expr(f"date_add('{start}', cast({rand_expr} * {days} as int))")

print("Helpers ready.")

# COMMAND ----------
# =====================================================================================
# CELL 4 — DIM: Employees
# =====================================================================================

banner("BUILD: dqx.demo_employee")

n_emp = ROW_TARGETS[TABLES["employee"]]
base = (spark.range(n_emp)
        .withColumn("_rn", F.row_number().over(Window.orderBy(F.monotonically_increasing_id())) - 1)
        .withColumn("_rand", F.rand(seed=42))
        .withColumn("_rand2", F.rand(seed=99)))

emp_df = base.select(
    seq_id("E", 1001).alias("employee_id"),
    F.concat(F.lit("Emp "), F.col("_rn")).alias("full_name"),
    choice_expr(["Consulting","Audit","Tax","IT","Ops"], [0.35,0.2,0.2,0.15,0.1]).alias("department"),
    choice_expr(["Engineer","Analyst","Consultant","Manager","Support"]).alias("role"),
    F.when(F.rand() < 0.02, F.lit(None)).otherwise(F.concat(F.lit("CC-"), F.lpad((F.rand()*9999).cast("int").cast("string"), 4, "0"))).alias("cost_center"),
    choice_expr(["Active","Leave","Terminated"], [0.90,0.03,0.07], seed_col="_rand2").alias("employment_status"),
    rand_date("2018-01-01", "2025-08-01", "rand()").alias("hire_date"),
    F.lit(None).cast("date").alias("termination_date"),
    F.concat(F.lit("emp"), F.col("_rn").cast("string"), F.lit("@company.com")).alias("work_email"),
    choice_expr(["US","CA","MX","GB","IN"], [0.5,0.15,0.1,0.15,0.1]).alias("country_code")
)

emp_df = (emp_df
    # inject missing/invalid termination_date for a slice of Terminated
    .withColumn("termination_date",
        F.when((F.col("employment_status")=="Terminated") & (F.rand() < 0.08), F.lit(None).cast("date"))
         .otherwise(F.when(F.col("employment_status")=="Terminated",
                           F.expr("date_add(hire_date, cast(rand()*2500 + 30 as int))"))
                    .otherwise(F.lit(None).cast("date")))
    )
    .withColumn("work_email",
        F.when(F.rand() < 0.03, F.concat(F.lit("emp"), F.col("employee_id").substr(F.length(F.col("employee_id"))-3,4), F.lit("@gmail.com")))
         .otherwise(F.col("work_email"))
    )
    .withColumn("country_code",
        F.when(F.rand() < 0.005, F.lit("XX")).otherwise(F.col("country_code"))
    )
)

emp_df.write.format("delta").mode("overwrite").saveAsTable(TABLES["employee"])
print(f"Wrote: {TABLES['employee']}  rows={spark.table(TABLES['employee']).count():,}")
display(spark.table(TABLES["employee"]).limit(5))

# COMMAND ----------
# =====================================================================================
# CELL 5 — DIM: Customers
# =====================================================================================

banner("BUILD: dqx.demo_customer")

n_cust = ROW_TARGETS[TABLES["customer"]]
base = spark.range(n_cust).withColumn("_rn", F.row_number().over(Window.orderBy(F.monotonically_increasing_id())) - 1)

cust_df = base.select(
    seq_id("C", 5001).alias("customer_id"),
    F.concat(F.lit("Customer "), F.col("_rn")).alias("customer_name"),
    choice_expr(["Technology","Healthcare","Finance","Retail","Manufacturing"], [0.3,0.2,0.2,0.2,0.1]).alias("industry"),
    choice_expr(["US","CA","MX","GB","IN"]).alias("country_code"),
    choice_expr(["Active","Prospect","Inactive"], [0.75,0.15,0.10]).alias("status"),
    rand_date("2019-01-01", "2025-08-01", "rand()").alias("onboarding_date"),
    F.when(F.rand() < 0.02, F.concat(F.lit("contact"), F.col("_rn").cast("string")))
     .otherwise(F.concat(F.lit("contact"), F.col("_rn").cast("string"), F.lit("@example.com"))).alias("primary_contact_email"),
    F.concat(F.lit("RN-"), F.lpad((F.rand()*99999999).cast("int").cast("string"), 8, "0")).alias("registration_number")
)

# duplicate registration_number among Actives (~1%)
dupes = cust_df.where(F.col("status")=="Active").limit(int(n_cust*0.01)).select(
    F.col("registration_number").alias("dup_reg")
)
cust_df = (cust_df.join(dupes.limit(1), how="left")
           .withColumn("registration_number",
               F.when(F.rand()<0.01, F.col("dup_reg")).otherwise(F.col("registration_number")))
           .drop("dup_reg","_rn"))

cust_df.write.format("delta").mode("overwrite").saveAsTable(TABLES["customer"])
print(f"Wrote: {TABLES['customer']}  rows={spark.table(TABLES['customer']).count():,}")
display(spark.table(TABLES["customer"]).limit(5))

# COMMAND ----------
# =====================================================================================
# CELL 6 — DIM: Projects (FKs to Customer & Employee)
# =====================================================================================

banner("BUILD: dqx.demo_project")

n_proj = ROW_TARGETS[TABLES["project"]]
emp_lu = spark.table(TABLES["employee"]).select("employee_id").withColumn("emp_idx", F.row_number().over(Window.orderBy("employee_id")) - 1)
cust_lu = spark.table(TABLES["customer"]).select("customer_id").withColumn("cust_idx", F.row_number().over(Window.orderBy("customer_id")) - 1)
emp_count, cust_count = emp_lu.count(), cust_lu.count()

proj_base = spark.range(n_proj).withColumn("_rn", F.row_number().over(Window.orderBy(F.monotonically_increasing_id())) - 1)

proj_df = (proj_base.select(
    seq_id("P", 10001).alias("project_id"),
    (F.col("_rn") % cust_count).alias("cust_idx"),
    F.concat(F.lit("Project "), F.col("_rn")).alias("project_name"),
    choice_expr(["Planned","Active","OnHold","Closed"], [0.15,0.55,0.10,0.20]).alias("status"),
    rand_date("2020-01-01", "2025-03-01", "rand()").alias("start_date"),
    F.lit(None).cast("date").alias("end_date"),
    (F.col("_rn") % emp_count).alias("emp_idx"),
    (F.rand()* (2_000_000 - 10_000) + 10_000).cast("decimal(18,2)").alias("budget_amount"),
    choice_expr(["T&M","Fixed","Retainer"], [0.55,0.35,0.10]).alias("billing_model")
).join(cust_lu, "cust_idx", "left").join(emp_lu, "emp_idx", "left")
 .withColumn("end_date", F.when(F.rand() < 0.65, F.expr("date_add(start_date, cast(rand()*1200 + 30 as int))")).otherwise(F.lit(None).cast("date")))
 .withColumn("budget_amount", F.when(F.rand() < 0.004, F.lit(0).cast("decimal(18,2)")).otherwise(F.col("budget_amount")))
 .select("project_id","customer_id","project_name","status","start_date","end_date","employee_id","budget_amount","billing_model")
 .withColumnRenamed("employee_id","manager_employee_id")
 .withColumn("end_date", F.when(F.rand()<0.007, F.expr("date_add(start_date, -cast(rand()*90 + 1 as int))")).otherwise(F.col("end_date")))
)

proj_df.write.format("delta").mode("overwrite").saveAsTable(TABLES["project"])
print(f"Wrote: {TABLES['project']}  rows={spark.table(TABLES['project']).count():,}")
display(spark.table(TABLES["project"]).limit(5))

# COMMAND ----------
# =====================================================================================
# CELL 7 — FACT: Timesheets (FKs to Employee & Project)
# =====================================================================================

banner("BUILD: dqx.demo_timesheet")

n_ts = ROW_TARGETS[TABLES["timesheet"]]
emp_lu = spark.table(TABLES["employee"]).select("employee_id","employment_status").withColumn("emp_idx", F.row_number().over(Window.orderBy("employee_id")) - 1)
proj_lu = spark.table(TABLES["project"]).select("project_id","start_date","end_date").withColumn("proj_idx", F.row_number().over(Window.orderBy("project_id")) - 1)
emp_count, proj_count = emp_lu.count(), proj_lu.count()

ts_base = spark.range(n_ts).withColumn("_rn", F.row_number().over(Window.orderBy(F.monotonically_increasing_id())) - 1)

ts_df = (ts_base.select(
    F.expr("uuid()").alias("timesheet_id"),
    (F.col("_rn") % emp_count).alias("emp_idx"),
    ((F.col("_rn") * 13) % proj_count).alias("proj_idx"),
    rand_date("2024-01-01","2025-08-10","rand()").alias("work_date"),
    (F.round(F.rand()*12,2)).cast("decimal(5,2)").alias("hours_worked"),
    choice_expr(["Billable","NonBillable","Admin"], [0.70,0.25,0.05]).alias("work_type"),
    choice_expr(["Workday","Jira","CSV"], [0.70,0.20,0.10]).alias("source_system"),
    F.current_timestamp().alias("created_ts")
).join(emp_lu.select("employee_id","emp_idx"), "emp_idx", "left")
 .join(proj_lu.select("project_id","proj_idx","start_date","end_date"), "proj_idx", "left")
 .drop("emp_idx","proj_idx")
 .withColumn("work_date", F.when(F.rand()<0.003, F.expr("date_add(current_date(), cast(rand()*5 + 1 as int))")).otherwise(F.col("work_date")))
 .withColumn("hours_worked", F.when(F.rand()<0.003, F.when(F.rand()<0.5, F.lit(-1.0)).otherwise(F.lit(25.0))).otherwise(F.col("hours_worked")))
)

# inject duplicates on (employee_id, project_id, work_date) (~0.6%)
dupe_fraction = 0.006
ts_df = ts_df.unionByName(ts_df.sample(withReplacement=True, fraction=dupe_fraction, seed=11))

ts_df.write.format("delta").mode("overwrite").saveAsTable(TABLES["timesheet"])
print(f"Wrote: {TABLES['timesheet']}  rows={spark.table(TABLES['timesheet']).count():,}")
display(spark.table(TABLES["timesheet"]).limit(5))

# COMMAND ----------
# =====================================================================================
# CELL 8 — FACT: Expenses (FKs to Employee & Project)
# =====================================================================================

banner("BUILD: dqx.demo_expense")

n_exp = ROW_TARGETS[TABLES["expense"]]
emp_lu = spark.table(TABLES["employee"]).select("employee_id","employment_status").withColumn("emp_idx", F.row_number().over(Window.orderBy("employee_id")) - 1)
proj_lu = spark.table(TABLES["project"]).select("project_id","start_date","end_date").withColumn("proj_idx", F.row_number().over(Window.orderBy("project_id")) - 1)
emp_count, proj_count = emp_lu.count(), proj_lu.count()

exp_base = spark.range(n_exp).withColumn("_rn", F.row_number().over(Window.orderBy(F.monotonically_increasing_id())) - 1)

exp_df = (exp_base.select(
    F.expr("uuid()").alias("expense_id"),
    (F.col("_rn") % emp_count).alias("emp_idx"),
    F.when(F.rand()<0.25, F.lit(None)).otherwise(((F.col("_rn") * 17) % proj_count)).alias("proj_idx"),
    rand_date("2024-01-01","2025-08-10","rand()").alias("expense_date"),
    choice_expr(["Meals","Travel","Supplies","Software","Other"], [0.35,0.25,0.20,0.10,0.10]).alias("category"),
    (F.round((F.rand()**0.3) * 5000, 2)).cast("decimal(18,2)").alias("amount"),
    choice_expr(["USD","CAD","MXN","GBP","INR"]).alias("currency_code"),
    choice_expr(["Uber","Lyft","Delta","AA","Staples","BestBuy","Amazon","LocalCafe","HotelCo","SoftwareCo"]).alias("merchant"),
    (F.rand() > 0.08).alias("receipt_attached"),
    F.current_timestamp().alias("submission_ts")
).join(emp_lu.select("employee_id","emp_idx"), "emp_idx", "left")
 .join(proj_lu.select("project_id","proj_idx","start_date","end_date"), "proj_idx", "left")
 .drop("emp_idx","proj_idx")
 .withColumn("currency_code", F.when(F.rand()<0.002, F.lit("XXX")).otherwise(F.col("currency_code")))
 .withColumn("receipt_attached",
    F.when((F.col("amount") >= MEGASPEC["knobs"]["receipt_threshold"]) & (F.rand() < 0.5), F.lit(False))
     .otherwise(F.col("receipt_attached")))
)

# build dup set on (employee_id, merchant, expense_date, amount) (~0.8%)
dup_frac = 0.008
dups = (exp_df.sample(withReplacement=True, fraction=dup_frac, seed=12)
        .select("employee_id","merchant","expense_date","amount")
        .withColumn("dup_key", F.lit(1)))
exp_df = (exp_df.join(dups.drop("dup_key"), ["employee_id","merchant","expense_date","amount"], "left")
               .unionByName(exp_df.where(F.col("dup_key").isNotNull()).drop("dup_key"))
               .withColumn("dup_key", F.lit(None).cast("int")))

exp_df.write.format("delta").mode("overwrite").saveAsTable(TABLES["expense"])
print(f"Wrote: {TABLES['expense']}  rows={spark.table(TABLES['expense']).count():,}")
display(spark.table(TABLES["expense"]).limit(5))

# COMMAND ----------
# =====================================================================================
# CELL 9 — Rules: default builder (used if your RULE_CLASSES are still stubs)
# =====================================================================================

banner("RULES: load from classes or fall back to default demo rules")

def build_default_rules():
    k = MEGASPEC["knobs"]
    valid_ccy = ",".join([f"'{c}'" for c in k["valid_currency_codes"]])
    weekend_level = MEGASPEC["flags"]["allow_weekend_billable"]
    weekend_level = "warn" if weekend_level not in ("warn","error") else weekend_level

    return {
      TABLES["employee"]: [
        {"id":"EMP_TERM_DATE_MISSING","level":"error","message":"Terminated missing termination_date",
         "expr":"employment_status = 'Terminated' AND termination_date IS NULL"},
        {"id":"EMP_TERM_DATE_BEFORE_HIRE","level":"error","message":"termination_date < hire_date",
         "expr":"employment_status = 'Terminated' AND termination_date < hire_date"},
        {"id":"EMP_EMAIL_DOMAIN_WARN","level":"warn","message":"Non-company or missing email",
         "expr":"work_email IS NULL OR NOT work_email LIKE '%@company.com'"},
      ],
      TABLES["customer"]: [
        {"id":"CUST_REG_DUP_ACTIVE","level":"error","message":"Duplicate registration_number among Active",
         "expr": f"""
            registration_number IS NOT NULL AND status = 'Active' AND registration_number IN (
              SELECT registration_number FROM {TABLES["customer"]}
              WHERE registration_number IS NOT NULL AND status = 'Active'
              GROUP BY registration_number HAVING COUNT(*) > 1
            )
         """},
      ],
      TABLES["project"]: [
        {"id":"PROJ_END_BEFORE_START","level":"error","message":"end_date before start_date",
         "expr":"end_date IS NOT NULL AND end_date < start_date"},
        {"id":"PROJ_FK_CUSTOMER_MISSING","level":"error","message":"customer_id not found",
         "expr":f"customer_id NOT IN (SELECT customer_id FROM {TABLES['customer']})"},
      ],
      TABLES["timesheet"]: [
        {"id":"TS_NEG_OR_GT24","level":"error","message":"Hours must be within 0..24",
         "expr":f"hours_worked < 0 OR hours_worked > {k['max_hours_per_day_error']}"},
        {"id":"TS_FUTURE_DATE","level":"error","message":"Work date in future",
         "expr":"work_date > current_date()"},
        {"id":"TS_EMP_STATUS_ERROR","level":"error","message":"Non-active employee",
         "expr":f"employee_id IN (SELECT employee_id FROM {TABLES['employee']} WHERE employment_status <> 'Active')"},
        {"id":"TS_DUP_EMP_PROJ_DAY","level":"warn","message":"Duplicate emp+proj+day",
         "expr":f"""
           (employee_id, project_id, work_date) IN (
             SELECT employee_id, project_id, work_date
             FROM {TABLES['timesheet']}
             GROUP BY employee_id, project_id, work_date
             HAVING COUNT(*) > 1
           )
         """},
        {"id":"TS_OUTSIDE_PROJECT_WINDOW","level":"error","message":"Work outside project window",
         "expr":f"""
           work_date < (SELECT start_date FROM {TABLES['project']} p WHERE p.project_id = {TABLES['timesheet']}.project_id)
           OR (
              (SELECT end_date FROM {TABLES['project']} p2 WHERE p2.project_id = {TABLES['timesheet']}.project_id) IS NOT NULL
              AND work_date >
                  (SELECT end_date FROM {TABLES['project']} p3 WHERE p3.project_id = {TABLES['timesheet']}.project_id)
           )
         """},
        {"id":"TS_WEEKEND_BILLABLE","level":weekend_level,"message":"Billable hours on weekend",
         "expr":"work_type = 'Billable' AND date_format(work_date, 'E') IN ('Sat','Sun')"},
      ],
      TABLES["expense"]: [
        {"id":"EXP_DUP","level":"error","message":"Duplicate expense",
         "expr":f"""
           (employee_id, merchant, expense_date, amount) IN (
             SELECT employee_id, merchant, expense_date, amount
             FROM {TABLES['expense']}
             GROUP BY employee_id, merchant, expense_date, amount
             HAVING COUNT(*) > 1
           )
         """},
        {"id":"EXP_NO_RECEIPT_OVER_T","level":"error","message":"Receipt required at/over threshold",
         "expr":f"amount >= {k['receipt_threshold']} AND receipt_attached = false"},
        {"id":"EXP_OOP","level":"warn","message":"Out-of-policy amount (Meals>limit or Travel>limit)",
         "expr":f"(category = 'Meals' AND amount > {k['meal_limit']}) OR (category = 'Travel' AND amount > {k['travel_limit']})"},
        {"id":"EXP_BAD_CCY","level":"error","message":"Invalid currency code",
         "expr":f"currency_code NOT IN ({valid_ccy})"},
        {"id":"EXP_EMP_STATUS_ERROR","level":"error","message":"Non-active employee",
         "expr":f"employee_id IN (SELECT employee_id FROM {TABLES['employee']} WHERE employment_status <> 'Active')"},
        {"id":"EXP_OUTSIDE_PROJECT_WINDOW","level":"error","message":"Expense outside project window",
         "expr":f"""
           project_id IS NOT NULL AND (
             expense_date < (SELECT start_date FROM {TABLES['project']} p WHERE p.project_id = {TABLES['expense']}.project_id)
             OR (
               (SELECT end_date FROM {TABLES['project']} p2 WHERE p2.project_id = {TABLES['expense']}.project_id) IS NOT NULL
               AND expense_date >
                   (SELECT end_date FROM {TABLES['project']} p3 WHERE p3.project_id = {TABLES['expense']}.project_id)
             )
           )
         """},
      ]
    }

def load_rules():
    rules_by_table = {}
    for tbl, cls in RULE_CLASSES_BY_TABLE.items():
        try:
            rset = cls()
            rules = rset.rules()  # should return list[dict]
            rules_by_table[tbl] = rules or []
        except Exception:
            rules_by_table[tbl] = []
    defaults = build_default_rules()
    for tbl in defaults:
        if tbl not in rules_by_table or not rules_by_table[tbl]:
            rules_by_table[tbl] = defaults[tbl]
    return rules_by_table

RULES = load_rules()
print("Rules loaded for:")
for k in TABLES["sources"]:
    print("  •", k, f"({len(RULES.get(k, []))} rules)")

# COMMAND ----------
# =====================================================================================
# CELL 10 — Rule Runner (append warning/error, write quarantine, keep clean rows)
# =====================================================================================

banner("APPLY: Run rules per table, quarantine errors, keep warned rows")

def apply_rules_to_table(table_fullname: str, rules: list, run_id: str):
    if not rules:
        print(f"[{table_fullname}] No rules; skipping.")
        return

    errs  = [r for r in rules if r.get("level","").lower()=="error"]
    warns = [r for r in rules if r.get("level","").lower()=="warn"]

    def _make_array_expr(rule_list):
        if not rule_list:
            return "array()"
        parts = [f"IF({r['expr']}, '{r['id']}', NULL)" for r in rule_list]
        return f"array_remove(array({', '.join(parts)}), NULL)"

    error_arr_sql = _make_array_expr(errs)
    warn_arr_sql  = _make_array_expr(warns)

    flagged_sql = f"""
      SELECT
        t.*,
        {warn_arr_sql}  AS warning,
        {error_arr_sql} AS error
      FROM {table_fullname} t
    """
    flagged = spark.sql(flagged_sql)

    if errs:
        err_map = spark.createDataFrame(
            [(e["id"], e.get("message","")) for e in errs],
            "rule_id STRING, message STRING"
        )
        error_rows = flagged.where(F.size(F.col("error")) > 0)
        exploded = (error_rows.select(
            F.lit(table_fullname).alias("_source_table"),
            F.explode(F.col("error")).alias("_rule_id"),
            F.lit("ERROR").alias("_level"),
            F.lit(run_id).alias("_run_id"),
            F.current_timestamp().alias("_event_ts"),
            F.to_json(F.struct([F.col(c) for c in flagged.columns])).alias("_row_payload_json")
        ).join(err_map, F.col("_rule_id")==F.col("rule_id"), "left")
         .select("_source_table","_rule_id","_level",F.col("message").alias("_reason"),"_run_id","_event_ts","_row_payload_json"))
        exploded.write.format("delta").mode("append").saveAsTable(QUARANTINE_TABLE)

    cleaned = flagged.where(F.size(F.col("error")) == 0)
    cleaned.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(table_fullname)

    total  = flagged.count()
    err_ct = flagged.where(F.size(F.col("error")) > 0).count()
    warn_ct= cleaned.where(F.size(F.col("warning")) > 0).count()
    print(f"[{table_fullname}] total={total:,}  quarantined={err_ct:,}  warned_on_kept={warn_ct:,}")

for tbl in TABLES["sources"]:
    apply_rules_to_table(tbl, RULES.get(tbl, []), run_id)

print("Done applying rules.")

# COMMAND ----------
# =====================================================================================
# CELL 11 — Demo Views (summaries you can show)
# =====================================================================================

banner("DEMO: Quarantine summary by rule")
display(spark.sql(f"""
SELECT _rule_id, COUNT(*) AS hits
FROM {QUARANTINE_TABLE}
WHERE _run_id = '{run_id}'
GROUP BY _rule_id
ORDER BY hits DESC
"""))

banner("DEMO: Which tables had warnings?")
for tbl in TABLES["sources"]:
    df = spark.table(tbl)
    if "warning" in df.columns:
        cnt = df.where(F.size(F.col("warning")) > 0).count()
        print(f"  {tbl}: {cnt:,} rows with warnings")

banner("DEMO: Sample warned rows (expenses)")
display(spark.sql(f"""
SELECT expense_id, employee_id, category, amount, currency_code, receipt_attached, warning
FROM {TABLES['expense']}
WHERE size(warning) > 0
LIMIT 20
"""))

banner("DEMO: Row counts after quarantine")
display(spark.sql(f"SHOW TABLES IN {DQX_SCHEMA}"))
for tbl in TABLES["sources"]:
    print(tbl, spark.table(tbl).count())

In [0]:
# COMMAND ----------
# =====================================================================================
# CELL 12 — Plant deterministic demo records (the "smoking guns")
# =====================================================================================
from pyspark.sql import functions as F

banner("PLANT: deterministic showcase rows & updates")

# -- 12.1 Make sure specific IDs exist (they do, given our generators):
# Employees E1001..E2999, Projects P10001..P10600, Customers C5001..C5999

# -- 12.2 Ensure E1099 is Terminated (to trigger EMP_STATUS rules)
spark.sql(f"""
UPDATE {TABLES['employee']}
SET employment_status = 'Terminated',
    termination_date  = '2024-06-15'
WHERE employee_id = 'E1099'
""")
print("Updated E1099 -> Terminated.")

# -- 12.3 Force a project window end to use in 'outside window' cases
spark.sql(f"""
UPDATE {TABLES['project']}
SET start_date = '2024-10-01',
    end_date   = '2025-02-28',
    status     = 'Closed'
WHERE project_id = 'P10037'
""")
print("Updated P10037 -> Closed, end=2025-02-28.")

# -- 12.4 Duplicate Active customer registration numbers
spark.sql(f"""
UPDATE {TABLES['customer']}
SET status='Active', registration_number='RN-00001234'
WHERE customer_id IN ('C5009','C5017')
""")
print("Set C5009/C5017 to Active with same registration_number.")

# -- 12.5 Insert planted TIMESHEETS
spark.sql(f"""
INSERT INTO {TABLES['timesheet']} (timesheet_id, employee_id, project_id, work_date, hours_worked,
                                    work_type, source_system, created_ts)
VALUES
  ('TS_DEMO_001','E1015','P10012','2025-07-10',25.00,'Billable','CSV',current_timestamp()),    -- TS_NEG_OR_GT24
  ('TS_DEMO_002','E1044','P10012','2025-07-13', 8.00,'Billable','CSV',current_timestamp()),    -- TS_WEEKEND_BILLABLE (Sun)
  ('TS_DEMO_003','E1010','P10012', date_add(current_date(), 3), 2.00,'Admin','CSV',current_timestamp()), -- TS_FUTURE_DATE
  ('TS_DEMO_004','E1099','P10012','2025-06-20', 3.50,'Billable','CSV',current_timestamp()),    -- TS_EMP_STATUS_ERROR
  ('TS_DEMO_005','E1010','P10037','2025-07-01', 6.00,'Billable','CSV',current_timestamp()),    -- TS_OUTSIDE_PROJECT_WINDOW
  ('TS_DEMO_006','E1033','P10012','2025-06-05', 4.00,'Billable','CSV',current_timestamp()),    -- TS_DUP pair A
  ('TS_DEMO_007','E1033','P10012','2025-06-05', 4.00,'Billable','CSV',current_timestamp())     -- TS_DUP pair B
""")
print("Inserted planted timesheets (TS_DEMO_001..007).")

# -- 12.6 Insert planted EXPENSES
spark.sql(f"""
INSERT INTO {TABLES['expense']} (expense_id, employee_id, project_id, expense_date, category, amount,
                                  currency_code, merchant, receipt_attached, submission_ts)
VALUES
  ('EX_DEMO_001','E1010','P10012','2025-07-15','Travel',399.99,'USD','HotelCo',true,current_timestamp()), -- EXP_DUP A
  ('EX_DEMO_002','E1010','P10012','2025-07-15','Travel',399.99,'USD','HotelCo',true,current_timestamp()), -- EXP_DUP B
  ('EX_DEMO_003','E1011','P10012','2025-07-16','Meals',220.00,'USD','LocalCafe',false,current_timestamp()), -- EXP_NO_RECEIPT_OVER_T
  ('EX_DEMO_004','E1012','P10012','2025-07-16','Supplies',42.00,'XXX','Staples',true,current_timestamp()),  -- EXP_BAD_CCY
  ('EX_DEMO_005','E1099','P10012','2025-07-10','Other',50.00,'USD','Amazon',true,current_timestamp()),      -- EXP_EMP_STATUS_ERROR
  ('EX_DEMO_006','E1013','P10012','2025-07-18','Travel',799.00,'USD','Delta',true,current_timestamp()),     -- EXP_OOP (warn)
  ('EX_DEMO_007','E1014','P10037','2025-07-20','Software',123.45,'USD','SoftwareCo',true,current_timestamp()) -- EXP_OUTSIDE_PROJECT_WINDOW
""")
print("Inserted planted expenses (EX_DEMO_001..007).")

print("Planting complete.")

# COMMAND ----------
# =====================================================================================
# CELL 13 — Re-apply rules to reflect planted rows (quarantine + warnings)
# =====================================================================================

banner("RE-APPLY: Apply rules to impacted tables after planting")

# Re-load RULES in case you edited rule classes
RULES = load_rules()

# Re-apply to the specific tables we touched
for tbl in [TABLES["employee"], TABLES["customer"], TABLES["project"], TABLES["timesheet"], TABLES["expense"]]:
    apply_rules_to_table(tbl, RULES.get(tbl, []), run_id)

print("Re-apply complete.")

# COMMAND ----------
# =====================================================================================
# CELL 14 — Dashboard Views (build once, then wire to Databricks SQL dashboard)
# =====================================================================================

banner("DASHBOARD: Create/refresh views for the demo")

# 14.1 Quarantine by rule (for this run)
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW v_quarantine_by_rule AS
SELECT _rule_id, COUNT(*) AS hits
FROM {QUARANTINE_TABLE}
WHERE _run_id = '{run_id}'
GROUP BY _rule_id
ORDER BY hits DESC
""")

# 14.2 Quarantine detail (drill)
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW v_quarantine_detail AS
SELECT
  _source_table,
  _rule_id,
  _level,
  _reason,
  _run_id,
  _event_ts,
  _row_payload_json
FROM {QUARANTINE_TABLE}
WHERE _run_id = '{run_id}'
ORDER BY _event_ts DESC
""")

# 14.3 Warnings by table (rows that passed but were marked)
warn_counts = []
for tbl in TABLES["sources"]:
    if "warning" in spark.table(tbl).columns:
        cnt = spark.table(tbl).where(F.size(F.col("warning")) > 0).count()
        warn_counts.append((tbl, cnt))
warn_df = spark.createDataFrame(warn_counts, "table STRING, warn_rows BIGINT")
warn_df.createOrReplaceTempView("v_warning_counts")

# 14.4 Top entities to triage
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW v_top_employees_quarantined AS
SELECT
  payload.employee_id AS employee_id,
  COUNT(*) AS error_hits
FROM (
  SELECT from_json(_row_payload_json, schema_of_json(_row_payload_json)) AS payload
  FROM {QUARANTINE_TABLE}
  WHERE _run_id = '{run_id}'
) q
GROUP BY payload.employee_id
ORDER BY error_hits DESC
""")

spark.sql(f"""
CREATE OR REPLACE TEMP VIEW v_top_merchants_quarantined AS
SELECT
  payload.merchant AS merchant,
  COUNT(*) AS error_hits
FROM (
  SELECT from_json(_row_payload_json, schema_of_json(_row_payload_json)) AS payload
  FROM {QUARANTINE_TABLE}
  WHERE _run_id = '{run_id}'
) q
WHERE payload.merchant IS NOT NULL
GROUP BY payload.merchant
ORDER BY error_hits DESC
""")

# 14.5 Handy spotlight views for the planted cases
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW v_demo_cases AS
SELECT
  _rule_id,
  _source_table,
  _reason,
  _event_ts,
  get_json_object(_row_payload_json, '$.employee_id')    AS employee_id,
  get_json_object(_row_payload_json, '$.project_id')     AS project_id,
  get_json_object(_row_payload_json, '$.work_date')      AS work_date,
  get_json_object(_row_payload_json, '$.hours_worked')   AS hours_worked,
  get_json_object(_row_payload_json, '$.expense_date')   AS expense_date,
  get_json_object(_row_payload_json, '$.category')       AS category,
  get_json_object(_row_payload_json, '$.merchant')       AS merchant,
  get_json_object(_row_payload_json, '$.amount')         AS amount
FROM {QUARANTINE_TABLE}
WHERE _run_id = '{run_id}'
  AND (
        (_rule_id IN ('EXP_DUP','EXP_NO_RECEIPT_OVER_T','EXP_BAD_CCY','EXP_EMP_STATUS_ERROR','EXP_OUTSIDE_PROJECT_WINDOW'))
     OR (_rule_id IN ('TS_NEG_OR_GT24','TS_FUTURE_DATE','TS_OUTSIDE_PROJECT_WINDOW','TS_EMP_STATUS_ERROR'))
     OR (_rule_id = 'CUST_REG_DUP_ACTIVE')
  )
ORDER BY _event_ts DESC
""")

print("Views created:")
print("  - v_quarantine_by_rule")
print("  - v_quarantine_detail")
print("  - v_warning_counts")
print("  - v_top_employees_quarantined")
print("  - v_top_merchants_quarantined")
print("  - v_demo_cases")

banner("DASHBOARD QUICK LOOK")
print("Quarantine by rule:")
display(spark.sql("SELECT * FROM v_quarantine_by_rule"))

print("Warnings by table:")
display(spark.sql("SELECT * FROM v_warning_counts ORDER BY warn_rows DESC"))

print("Spotlight on our planted cases:")
display(spark.sql("SELECT * FROM v_demo_cases LIMIT 20"))